In [ ]:
import sys
sys.path.append('../')
sys.path.append('../helpers')
import pyVoxelFEM
from tri_mesh_viewer import QuadHexViewer
import MeshFEM, mesh
from ipopt_helpers import initializeTensorProductSimulator, problemObjectWrapper, initializeIpoptProblem
from debug_helpers import numericalDerivative, plotConvergence

import numpy as np
import matplotlib.pyplot as plt

# Filters tests
Run some iterations of TO (without AM constraints) to get a non trivial structure, then filter that structure and plot result.

In [ ]:
# Perform some iterations of the optimization with a TotalVolumeConstintProblem to get a non trivial density filed

# Parameters
MATERIAL_PATH = '../../examples/materials/B9Creator.material'
BC_PATH = '../../examples/bcs/cantilever_flexion_E.bc'
orderFEM = [1, 1]
domainCorners = [[0, 0], [2, 1]]
gridDimensions = [60, 30]
E0 = 1
Emin = 1e-9
SIMPExponent = 3
maxVolume = 0.6
constraints = [pyVoxelFEM.TotalVolumeConstraint(maxVolume)]
filters = [pyVoxelFEM.SmoothingFilter(), pyVoxelFEM.ProjectionFilter()]
uniformDensity = maxVolume

# Initialize TO problem
tps = initializeTensorProductSimulator(
    orderFEM, domainCorners, gridDimensions, uniformDensity, E0, Emin, SIMPExponent, MATERIAL_PATH, BC_PATH
)
objective = pyVoxelFEM.ComplianceObjective(tps)
top = pyVoxelFEM.TopologyOptimizationProblem(tps, objective, constraints, filters)
nonLinearProblem, history = initializeIpoptProblem(top)
x0 = tps.getDensities()

# Run some iterations to get a non trivial density field
nonLinearProblem.addOption('max_iter', 30)
nonLinearProblem.addOption('tol', 1e-12)
x0, info = nonLinearProblem.solve(x0)

view = QuadHexViewer(*tps.getMesh(), scalarField=np.ones(x0.shape))
view.show()

In [ ]:
# Reference density field
view.update(scalarField=history.density[-1])

In [ ]:
# Smoothing filter
testRadius = 1
smoothFilt = pyVoxelFEM.SmoothingFilter()
pyVoxelFEM.TopologyOptimizationProblem(tps, objective, constraints, [smoothFilt]) # let TOP initialize filter's grid dimensions
smoothFilt.radius = testRadius
view.update(scalarField=pyVoxelFEM.applyFilter(smoothFilt, history.density[-1]))

In [ ]:
# Projection filter
testBeta = 4
projFilt = pyVoxelFEM.ProjectionFilter()
pyVoxelFEM.TopologyOptimizationProblem(tps, objective, constraints, [projFilt]) # let TOP initialize filter's grid dimensions
projFilt.beta = testBeta
view.update(scalarField=pyVoxelFEM.applyFilter(projFilt, history.density[-1]))

In [ ]:
# Langelaar's filter
langFilt = pyVoxelFEM.LangelaarFilter()
pyVoxelFEM.TopologyOptimizationProblem(tps, objective, constraints, [langFilt]) # let TOP initialize filter's grid dimensions
view.update(scalarField=pyVoxelFEM.applyFilter(langFilt, history.density[-1]))

In [ ]:
# Export filtered densities
mfw = mesh.MSHFieldWriter('../gmsh/Filters.msh', *tps.getMesh())
mfw.addField('NoFilter', history.density[-1])
mfw.addField('Smoothing', pyVoxelFEM.applyFilter(smoothFilt, history.density[-1]))
mfw.addField('Projection', pyVoxelFEM.applyFilter(projFilt, history.density[-1]))
mfw.addField('Langelaar', pyVoxelFEM.applyFilter(langFilt, history.density[-1]))
del mfw